# Read SpaceX Launch Data from https://api.spacexdata.com/v4

Documentation of the SpaceX API can be found @ https://github.com/r-spacex/SpaceX-API/tree/master/docs

In [ ]:
import requests

# Pick a dataset and the appropriate url (will require url+"<data>" in other calls to pull specific data)
url = "https://api.spacexdata.com/v4/"
# Get launch data
launches = requests.get(url+"launches").json()
# Get payload data
payloads = requests.get(url+"payloads").json()
# Get rockets data
rockets = requests.get(url+"rockets").json()

In [ ]:
print(f"There are {len(launches)} launches")
print(f"There are {len(payloads)} payloads")
print(f"There are {len(rockets)} rockets")

In [ ]:
# for launch in launches:
#   print(launch.get('name'), launch.get("id"), launch.get('date_utc'), launch.get("rocket"))

In [ ]:
# for rocket in rockets:
#   print(rocket.get("name"), rocket.get("id"), rocket.get("type"), rocket.get("cost_per_launch"))

In [131]:
# for payload in payloads:
  # print(payload.get("name"), payload.get("id"), payload.get("type"), payload.get("launch"), payload.get("mass_kg"))

In [101]:
import os
import sqlite3

# Delete the old database file if it exists
if os.path.exists("spacex.db"):
    os.remove("spacex.db")
    print("spacex.db database deleted")

spacex.db database deleted


In [104]:
# Create a connection (this creates spacex.db in your current Colab environment)
conn = sqlite3.connect("spacex.db")

# Create a cursor
cur = conn.cursor()

schema = """
CREATE TABLE IF NOT EXISTS rockets (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    cost_per_launch INTEGER

);

CREATE TABLE IF NOT EXISTS launches (
    id TEXT PRIMARY KEY,
    name TEXT,
    date_utc TEXT,
    rocket_id TEXT
);

CREATE TABLE IF NOT EXISTS payloads (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    mass_kg INTEGER,
    launch_id TEXT
);
"""

cur.executescript(schema)
conn.commit()
print("Database schema created")

Database schema created


In [105]:
# Insert data from api into rockets table, with only the columns we want
# ignore preventing pk conflicts
for r in rockets:
    cur.execute("""
        INSERT OR IGNORE INTO rockets
        (id, name, type, cost_per_launch)
        VALUES (?, ?, ?, ?)
    """, (
        r["id"], r["name"], r["type"], r["cost_per_launch"]
    ))

conn.commit()
print("Rockets data inserted")

Rockets data inserted


In [106]:
# Insert data from api into launches table, with only the columns we want
# ignore preventing pk conflicts
for l in launches:
    cur.execute("""
        INSERT OR IGNORE INTO launches
        (id, name, date_utc, rocket_id)
        VALUES (?, ?, ?, ?)
    """, (
        l["id"], l["name"], l["date_utc"], l["rocket"]
    ))

conn.commit()
print("Launches data inserted")

Launches data inserted


In [109]:
# Insert data from api into payloads table, with only the columns we want
# ignore preventing pk conflicts
for p in payloads:
    cur.execute("""
        INSERT OR IGNORE INTO payloads
        (id, name, type, mass_kg, launch_id)
        VALUES (?, ?, ?, ?, ?)
    """, (
        p["id"], p["name"], p["type"], p["mass_kg"], p["launch"]
    ))

conn.commit()
print("Payloads data inserted")

Payloads data inserted


In [126]:
# Which rocket has cost spacex the most to launch over its lifetime?
df = pd.read_sql_query('''
WITH launch_count AS (
  SELECT count(*) as num_launch, rocket_id
  FROM launches
  GROUP BY rocket_id
)

SELECT r.name, (lc.num_launch*r.cost_per_launch) as total_launches_cost
FROM launch_count lc
LEFT JOIN rockets r
ON lc.rocket_id = r.id
ORDER BY total_launches_cost DESC
''', conn)
df

,name,total_launches_cost,num_launch
0,Falcon 9,9750000000,195
1,Falcon Heavy,450000000,5
2,Falcon 1,33500000,5


In [129]:
import pandas as pd

# Display table as a dataframe
df = pd.read_sql_query("SELECT * FROM payloads LIMIT 5;", conn)
df

,id,name,type,mass_kg,launch_id
0,5eb0e4b5b6c3bb0006eeb1e1,FalconSAT-2,Satellite,20.0,5eb87cd9ffd86e000604b32a
1,5eb0e4b6b6c3bb0006eeb1e2,DemoSAT,Satellite,NaN,5eb87cdaffd86e000604b32b
2,5eb0e4b6b6c3bb0006eeb1e3,Trailblazer,Satellite,NaN,5eb87cdbffd86e000604b32c
3,5eb0e4b6b6c3bb0006eeb1e4,PRESat,Satellite,NaN,5eb87cdbffd86e000604b32c
4,5eb0e4b7b6c3bb0006eeb1e5,RatSat,Satellite,165.0,5eb87cdbffd86e000604b32d
